### Shannon Envelope Assignment
#### Name: Armand Faris A Surbakti
#### NRP: 5023201051

#### Import Libraries First

In [3]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import threading

### Read Data and Plot Original PCG data

In [4]:
pcg = pd.read_csv("Print_16_v2_PCG_RV.txt",delimiter='\t')
pcg = pcg.iloc[:, 1].values
N = len(pcg)
Fs = 1000

fig = go.Figure(data=go.Scatter(x=np.arange(N)/Fs, y=pcg, mode="lines",
                                name="PCG", line=dict(color='red')))

fig.update_xaxes(title_text="Time (s)")
fig.update_yaxes(title_text="Amplitude")
fig.update_layout(title="Original PCG Data")

fig.show()

### Preparing Data for DFT (Discrete Fourier Transformation)

In [5]:
#prepare imaginary numbers
real = np.zeros(N)
imaj = np.zeros(N)
mag = np.zeros(N)

#create the DFT function based on equations
m = int(N/2)
def DFT():
    for k in range(N):
        real[k] = 0
        imaj[k] = 0
        for n in range(N):
            real[k] = real[k] + (pcg[n] * np.cos(2*np.pi*k*(n/N)))
            imaj[k] = imaj[k] - (pcg[n] * np.sin(2*np.pi*k*(n/N)))
        mag[k] = np.sqrt((real[k])**2 + (imaj[k])**2)
    mag[0] = mag[0] / 4

DFT()
bar_trace = go.Bar(
    x=np.arange(N//2),
    y=mag,
    name='DFT Magnitude Spectrum',
    marker=dict(
        color='blue',  
    )
)

layout = go.Layout(
    title='DFT Magnitude Spectrum',
    xaxis=dict(title='Frequency (Hz)'),
    yaxis=dict(title='Magnitude'),
    plot_bgcolor='white',  
    paper_bgcolor='white'  
)

#create and plot the plotly figure
fig = go.Figure(data=[bar_trace], layout=layout)
fig.show()

### After DFT, do Continuous Wavelet Transform
### Plot 2D Contour Plot

In [6]:
#Define the parameters and variables
row, col = 100, 100
a, da = 0.0001, 0.0009
dt = 1 / Fs
db = ((N - 1) * dt) / col
t = np.arange(N) * dt
f0 = 0.849
w0 = 2 * np.pi * f0

# Create the CWT function
def cwt(input, arg, Fs=Fs, row=row, col=col, a=a, da=da, db=db, t=t, f0=f0, w0=w0):
    global cwt_re, cwt_im
    cwt_re = np.zeros(shape=(row, col))
    cwt_im = np.zeros(shape=(row, col))
    
    if arg == 'real':
        for i in range(row):
            b = 0
            for j in range(col):
                rem = 1 / np.sqrt(a) * 1 / np.power(np.pi, 0.25) * np.exp(-(t - b) / a * (t - b) / a / 2) * np.cos(w0 * (t - b) / a)
                cwt_re[i, j] = np.sum(input * rem)
                b += db
            a += da
    elif arg == 'imaginary':
        for i in range(row):
            b = 0
            for j in range(col):
                imm = 1 / np.sqrt(a) * 1 / np.power(np.pi, 0.25) * np.exp(-(t - b) / a * (t - b) / a / 2) * np.sin(w0 * (t - b) / a)
                cwt_im[i, j] = np.sum(input * imm)
                b += db
            a += da
            
#Compute CWT for ECG signal
pcg_signal = pcg  
t1 = threading.Thread(target=cwt, args=(pcg_signal, 'real'))
t2 = threading.Thread(target=cwt, args=(pcg_signal, 'imaginary'))
t1.start()
t2.start()
t1.join()
t2.join()
cwt_magnitude = np.sqrt(np.square(cwt_re) + np.square(cwt_im))
#plot the 2D Contour
fig = go.Figure(data=go.Contour(
    x=np.arange(col),  
    y=np.arange(row),  
    z=cwt_magnitude,  
    colorscale="Rainbow",
    colorbar=dict(title="CWT Amplitude")  
))

#Update the layout
fig.update_layout(
    width=1000,  
    height=600,  
    margin=dict(l=0, r=0, b=0, t=0),  
    xaxis_title="time",  
    yaxis_title="scale",  
    title="CWT Contour"  
)

#Show the figure
fig.show()

### Plot 3D Surface CWT Plot

In [11]:
#plot the 3D Contour
fig = go.Figure(data=go.Surface(
    x=np.arange(col),  
    y=np.arange(row),  
    z=cwt_magnitude,  
    colorscale="Rainbow",
    colorbar=dict(title="CWT Amplitude")  
))

#Update the layout
fig.update_layout(
    width=1000,  
    height=600,  
    margin=dict(l=0, r=0, b=0, t=0),  
    xaxis_title="time",  
    yaxis_title="scale",  
    title="CWT Contour"  
)

#Show the figure
fig.show()

### Scale Plotting of Time and Frequency Table

In [7]:
#dataframe table plotting of scale and frequency information
fk = np.zeros(row+1)
nk = np.zeros(row+1)
f0 = 0.849
a= 0.0001
da= 0.0009
for i in range(row , -1, -1):
    fk[i] = round((f0 / (a + (i * da))) ,4)

for i in range(row  , -1, -1):
    nk[i] = round(((N / 100) * i)/ 1000, 4)
scale_info = {
    'Scale': np.arange(1,row+1),
    'Frequency [Hz]': fk[1:],
    'Time [s]': nk[1:]
}
scale_df = pd.DataFrame(scale_info)

### Shannon Envelope Method

In [13]:
#Function to normalize the signal
def normalize_signal(signal):
    max_value = np.max(np.abs(signal))
    normalized_signal = signal / max_value
    return normalized_signal
#Function to calculate the average Shannon energy
def calculate_average_shannon_energy(signal, frame_length):
    N = len(signal)
    num_frames = N // frame_length
    energy = np.zeros(num_frames)
    epsilon = 1e-10  
    for i in range(num_frames):
        frame = signal[i * frame_length: (i + 1) * frame_length]
        energy[i] = -np.sum((frame + epsilon)**2 * np.log((frame + epsilon)**2))
    return energy
#Function to calculate the Shannon envelope
def calculate_shannon_envelope(signal, frame_length, threshold_coefficient):
    normalized_signal = normalize_signal(signal)
    average_energy = calculate_average_shannon_energy(normalized_signal, frame_length)
    mean_energy = np.mean(average_energy)
    std_energy = np.std(average_energy)
    threshold = mean_energy - threshold_coefficient * std_energy
    shannon_envelope = np.where(average_energy > threshold, average_energy, 0)
    print("Mean Shannon Energy:", mean_energy)
    print("Standard Deviation of Shannon Energy:", std_energy)
    return shannon_envelope
#initiate the shannon plotting
if __name__ == "__main__":
    heart_sound_signal = pcg  
    #Parameters
    frame_length = 20  
    threshold_coefficient = 0.1  
    #Calculate the Shannon envelope
    shannon_envelope = (calculate_shannon_envelope(heart_sound_signal, frame_length, threshold_coefficient))/10
    sample_rate = Fs  
    N = len(heart_sound_signal)
    time = (np.arange(N) / sample_rate)*20
    trace = go.Scatter(x=time, y=shannon_envelope, mode='lines', name='Shannon Envelope')
    layout = go.Layout(title='Shannon Envelope', xaxis=dict(title='Time (s)'), yaxis=dict(title='Amplitude'))
    fig = go.Figure(data=[trace], layout=layout)

    fig.show()
    


Mean Shannon Energy: 1.0236066623789588
Standard Deviation of Shannon Energy: 1.625330471890367


### Subplots Combination

In [17]:
fig = make_subplots(
    rows = 3, cols = 2,
    specs=[[{"type": "scatter"}, {"type": "scatter"}],
           [{"type": "contour"}, {"type": "surface"}],
           [{"type": "table"},{"type": "scatter"}]],
    vertical_spacing=0.10,
    subplot_titles=["PCG Signal", "DFT Output", "CWT Contour", "3D CWT Plot", "Scale and Time Table", "Segmented Signal"]
)
#Input the PCG Signal for plotting
fig.add_trace(go.Scatter(x=np.arange(N)/Fs, y=pcg,marker=dict(color='red'),mode="lines"), row=1, col=1)
fig.update_xaxes(title_text="Time (s)", row=1, col=1)
fig.update_yaxes(title_text="Amplitude", row=1, col=1)
#Input the DFT Signal for plotting
fig.add_trace(go.Bar(x=np.arange(N//2), y=mag,marker=dict(color='black')), row=1, col=2)
fig.update_xaxes(title_text="Frequency (Hz)", row=1, col=2)
fig.update_yaxes(title_text="Amplitude", row=1, col=2)
#2D CWT Contour Plot
contour = go.Contour(x=np.arange(col), 
                     y=np.arange(row), 
                     z=cwt_magnitude, 
                     colorscale="Rainbow",
                     colorbar=dict(title="CWT Amplitude"))  
fig.add_trace(contour, row=2, col=1)
fig.update_xaxes(title_text="Time", row=2, col=1)
fig.update_yaxes(title_text="Scale", row=2, col=1)
#3D CWT Surface Plot
fig.add_trace(go.Surface(x=np.arange(col), 
                         y=np.arange(row), 
                         z=cwt_magnitude, 
                         colorscale="Rainbow"), row=2, col=2)
fig.update_xaxes(title_text="Time", row=2, col=2)
fig.update_yaxes(title_text="Scale", row=2, col=2)

fig.update_layout(height=1000, width=1200, title_text="Plotting")

fig.update_layout(scene = dict(
                    xaxis_title = "Time Scale",
                    yaxis_title = "Frequency Scale",
                    zaxis_title = "Amplitude"
))
# Create a separate subplot for the table
table_trace = go.Table(header=dict(values=scale_df.columns),cells=dict(values=scale_df.values.T))
fig.add_trace(table_trace, row=3, col=1)
#Segmented Signal
trace1 = go.Scatter(x=np.arange(N) / Fs, y=pcg, mode='lines',marker=dict(color='red') ,name='PCG Signal')
trace2 = go.Scatter(x=time, y=shannon_envelope, mode='lines', marker=dict(color='green'), name='Shannon Envelope')
fig.add_trace(trace1, row=3, col=2)
fig.add_trace(trace2, row=3, col=2)
fig.update_xaxes(title_text="Time (s)", row=3, col=2)
fig.update_yaxes(title_text="Amplitude", row=3, col=2)
fig.show()

### Shannon Envelope Comparison

In [18]:
sample_rate = Fs  
N = len(heart_sound_signal)
time = (np.arange(N) / sample_rate)*20
trace1 = go.Scatter(x=np.arange(N)/Fs, y=pcg, mode='lines', line=dict(color='green'),name='Original PCG Data')
trace2 = go.Scatter(x=time, y=shannon_envelope, mode='lines', name='Shannon Envelope')
layout = go.Layout(title='Shannon Envelope Comparison', xaxis=dict(title='Time (s)'), yaxis=dict(title='Amplitude'))
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()


### Thresholding Contour

In [34]:
th_coef = 0.5  
max_cwt = np.max(cwt_magnitude)
min_cwt = np.min(cwt_magnitude)
th_mag = min_cwt + th_coef * max_cwt
cwt_thresholded = np.where(cwt_magnitude > th_mag, 1, 0)
fig = make_subplots(rows=1, cols=2, subplot_titles=["Original CWT", "Thresholded CWT"])
fig.add_trace(go.Contour(x=np.arange(col), y=np.arange(row), z=cwt_magnitude, colorscale="Rainbow", colorbar=dict(title="CWT Amplitude")), row=1, col=1)
fig.add_trace(go.Contour(x=np.arange(col), y=np.arange(row), z=cwt_thresholded, colorscale="Greys", showscale=False), row=1, col=2)
fig.update_layout(width=1200, height=400, margin=dict(l=0, r=0, b=0, t=0), xaxis_title="Time", yaxis_title="Scale", title="Original and Thresholded CWT Contour")
fig.show()